In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option("display.max_columns", None)
pd.set_option("display.max_info_columns", 150)
df = pd.read_csv("./stack-overflow-developer-survey-2024/survey_results_public.csv")

Let's start with some EDA.

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# The following columns are removed due to insufficient data or lack of relevance to our analysis.
df = df.drop(columns=['ResponseId', 'Check', 'Currency', 'CompTotal', 'NEWSOSites', 'SOVisitFreq', 'SOAccount', 
                      'SOPartFreq', 'SOHow', 'SOComm', 'AIToolNot interested in Using', 'AINextMuch more integrated', 
                      'AINextNo change', 'AINextMore integrated', 'AINextLess integrated', 'AINextMuch less integrated', 
                      'AIEthics', 'SurveyLength', 'SurveyEase'])


In [ ]:
# How is the group of participants composed?
df['MainBranch'].hist( xrot=80)

In [ ]:
df['Age'].hist( xrot=80)
# Most participants are between 18 and 44 years old.

In [ ]:
df['Employment'].describe()
# high number of unique values

In [ ]:
df['EdLevel'].hist( xrot=90)
# Most participants own a Bachelors or Maters degree

In [ ]:
df['RemoteWork'].hist( xrot=90)
# a large amount of people work at leat some amount remote

In [ ]:
df['ConvertedCompYearly'].hist( xrot=90, bins =500)
# The distribution of ConvertedCompYearly (annual compensation) is highly right-skewed, which is typical for salary data.

In [ ]:
df['JobSat'].hist( xrot=90, bins =10)
# Most respondents are either satisfied or neutral with their job. 
# Fewer respondents reported being very dissatisfied or very satisfied.
# The JobSat column has 65437 - 29126 = 36,311 missing values, which could represent nearly half of the dataset.
df['JobSat'].info()

In [ ]:
df['WorkExp'].hist( xrot=90, bins =10)
# The distribution of WorkExp (years of professional experience) is right-skewed, 
# meaning most respondents have relatively few years of experience, while a smaller number have many years.

There are several columns containing strings that can be easily translated into numerical values in a qualitative way. For this purpose, we create dictionaries and a function that performs the translation.

In [ ]:
selfdescription_dict = {'I am a developer by profession' : 5, 'I used to be a developer by profession, but no longer am': 4, 
                        'I am not primarily a developer, but I write code sometimes as part of my work/studies': 3, 
                        'I code primarily as a hobby':2, 'I am learning to code': 1, 'None of these': 0}
age_dict = {'Under 18 years old' : 0, '18-24 years old': 18, '25-34 years old': 25, '35-44 years old': 35, '45-54 years old': 45, 
            '55-64 years old': 55, '65 years or older': 65, 'Prefer not to say': np.nan }
education_dict = {'Professional degree (JD, MD, Ph.D, Ed.D, etc.)': 6, 'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)' : 5, 
                  'Bachelor’s degree (B.A., B.S., B.Eng., etc.)': 4, 'Associate degree (A.A., A.S., etc.)': 3, 
                  'Some college/university study without earning a degree':2, 
                  'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)': 1, 
                  'Primary/elementary school': 0, 'Something else': 3}
orgsize_dict = {'Just me - I am a freelancer, sole proprietor, etc.': 1, '2 to 9 employees' : 2, '10 to 19 employees': 10, 
                '20 to 99 employees': 20, '100 to 499 employees': 100, '500 to 999 employees': 500, '1,000 to 4,999 employees': 1000, 
                '5,000 to 9,999 employees': 5000, '10,000 or more employees': 10000, 'I don’t know': np.nan}
infl_dict = {'I have little or no influence': 0, 'I have some influence': 1, 'I have a great deal of influence': 2}
knowledge_dict = {'Agree':1, 'Strongly agree': 2,  'Neither agree nor disagree': 0,  'Disagree': -1, 
                  'Strongly disagree': -2, np.nan: 0}

def map_column_to_numbers(df, column, mapping_dict):
    """
    Replaces values in a DataFrame column using a given mapping dictionary.

    Args:
        df (pd.DataFrame): DataFrame containing the data.
        column (str): Name of the column to be mapped.
        mapping_dict (dict): Dictionary mapping strings to numerical values.

    Returns:
        pd.Series: The mapped column as a Pandas Series.
    """
    return df[column].replace(mapping_dict)

In [ ]:
df['MainBranch'] = map_column_to_numbers(df, 'MainBranch', selfdescription_dict)

df['Age'] = map_column_to_numbers(df, 'Age', age_dict)

df['EdLevel'] = map_column_to_numbers(df, 'EdLevel', education_dict)

df['OrgSize'] = map_column_to_numbers(df, 'OrgSize', orgsize_dict)

df['PurchaseInfluence'] = map_column_to_numbers(df, 'PurchaseInfluence', infl_dict)

for i in list(range(1, 10)): 
    df['Knowledge_'+str(i)] = map_column_to_numbers(df, 'Knowledge_'+str(i), knowledge_dict)

Year entries will be converted to integers where possible. If the entry is a string (e.g., indicating a range), it will be mapped to predefined boundary values.

In [ ]:
def clean_years_code(column):
    return column.replace({
        'Less than 1 year': 0,
        'More than 50 years': 60
    }).apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

df['YearsCode'] = clean_years_code(df['YearsCode'])
df['YearsCodePro'] = clean_years_code(df['YearsCodePro'])

Some columns contain information about tools that respondents use. These tools cannot be meaningfully ordered numerically. Creating dummy variables would significantly increase the dimensionality of the dataset and lead to sparse data. A more advanced approach would involve splitting the strings and filtering for the most relevant tools. However, we will choose a much simpler strategy here: we will count the number of tools mentioned by each respondent. This count might also serve as an indicator of job satisfaction.

In [ ]:
tools = ['LanguageHaveWorkedWith', 'LanguageWantToWorkWith', 'LanguageAdmired', 'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith', 
         'DatabaseAdmired', 'PlatformHaveWorkedWith', 'PlatformWantToWorkWith', 'PlatformAdmired', 'WebframeHaveWorkedWith', 
         'WebframeWantToWorkWith', 'WebframeAdmired', 'EmbeddedHaveWorkedWith', 'EmbeddedWantToWorkWith', 'EmbeddedAdmired', 
         'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith', 'MiscTechAdmired', 'ToolsTechHaveWorkedWith', 'ToolsTechWantToWorkWith', 
         'ToolsTechAdmired', 'NEWCollabToolsHaveWorkedWith', 'NEWCollabToolsWantToWorkWith', 'NEWCollabToolsAdmired', 
         'OpSysPersonal use', 'OpSysProfessional use', 'OfficeStackAsyncHaveWorkedWith', 'OfficeStackAsyncWantToWorkWith', 
         'OfficeStackAsyncAdmired', 'OfficeStackSyncHaveWorkedWith', 'OfficeStackSyncWantToWorkWith', 'OfficeStackSyncAdmired', 
         'AISearchDevHaveWorkedWith', 'AISearchDevWantToWorkWith', 'AISearchDevAdmired']

def count_tools_columns(df, columns):
    """
    Counts the number of tools (or entries) in specified columns 
    and creates new columns with the counts.

    Args:
        df (pd.DataFrame): The DataFrame containing the tool columns.
        columns (list): List of column names to process.

    Returns:
        pd.DataFrame: DataFrame with additional columns '<ColumnName>_Count'.
    """
    for col in columns:
        df[col] = df[col].apply(
            lambda x: str(x).count(';')+1 if pd.notnull(x) else 0
        )
    return df

count_tools_columns(df, tools)

We will now fill missing values where it is reasonable to do so.

In [ ]:
df.info()

In [ ]:
# The OrgSize-distribution is right-skewed. We replace missing values by the median.
df['OrgSize'].hist(bins = 100)
df['OrgSize'] = df['OrgSize'].fillna(df['OrgSize'].median())
# Missing values in influence are considered as little or no influence
df['PurchaseInfluence'] = df['PurchaseInfluence'].fillna(0)

In [ ]:
df0 = df[['MainBranch', 'Age', 'EdLevel', 'YearsCode', 'YearsCodePro', 'OrgSize', 'PurchaseInfluence', 'WorkExp', 'JobSatPoints_1',
         'JobSatPoints_4', 'JobSatPoints_5', 'JobSatPoints_6', 'JobSatPoints_7', 'JobSatPoints_8', 'JobSatPoints_9', 
          'JobSatPoints_10', 'JobSatPoints_11', 'ConvertedCompYearly', 'JobSat']]
corr = df0.corr(numeric_only=True)
plt.figure(figsize=(13, 10))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f", square=True)
plt.title("Correlation matrix")
plt.show()


In [ ]:
df1 = df[['YearsCode', 'YearsCodePro', 'OrgSize', 'PurchaseInfluence', 'WorkExp', 'Knowledge_1', 'Knowledge_2', 'Knowledge_3', 'Knowledge_4', 'Knowledge_5', 'Knowledge_6', 'Knowledge_7', 'Knowledge_8','JobSat']]
corr = df1.corr(numeric_only=True)
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f", square=True)
plt.title("Correlation matrix")
plt.show()

In [ ]:
df2 = df[['WorkExp', 'LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith', 'PlatformHaveWorkedWith', 'WebframeHaveWorkedWith', 
          'EmbeddedHaveWorkedWith', 'MiscTechHaveWorkedWith', 'ToolsTechHaveWorkedWith', 'NEWCollabToolsHaveWorkedWith', 
          'OpSysProfessional use', 'OfficeStackAsyncHaveWorkedWith', 'OfficeStackSyncHaveWorkedWith', 'AISearchDevHaveWorkedWith', 'JobSat']]
corr = df2.corr(numeric_only=True)
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f", square=True)
plt.title("Correlation matrix")
plt.show()

In [ ]:
df3 = df[['MainBranch', 'WorkExp', 'PurchaseInfluence', 'Knowledge_1', 'Knowledge_2', 'Knowledge_3', 'Knowledge_4', 'Knowledge_5', 'Knowledge_6', 'Knowledge_7', 'Knowledge_8', 'Knowledge_9', 'JobSat']]
corr = df3.corr(numeric_only=True)
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f", square=True)
plt.title("Correlation matrix")
plt.show()

In [ ]:
df['JobSat'].describe()
df.info()

In [ ]:
x = [65437-df['JobSat'].isna().sum(), df['JobSat'].isna().sum()]
labels = ['Job Saturation given', 'Job Saturation is NaN']

fig, ax = plt.subplots()
ax.pie(x, labels = labels)
ax.set_title('Survery responses')
plt.show()

In [ ]:
df['JobSat'].value_counts().sort_index().plot(kind='bar')
plt.xlabel('Job Satisfaction')
plt.ylabel('absolute frequency')
plt.show()

df['JobSat'].hist(bins=11)